# Los MiserAIbles
Sorany Hincapie Salazar  
Brayan Montoya Osorio

### Exploración de los datos

In [2]:
import pandas as pd

In [3]:
path_to_data = 'data/challenge_data-18-ago.csv'

df = pd.read_csv(path_to_data, sep = ';')
df.head(10)

,title,abstract,group
0,Adrenoleukodystrophy: survey of 303 cases: bio...,Adrenoleukodystrophy ( ALD ) is a genetically ...,neurological|hepatorenal
1,endoscopy reveals ventricular tachycardia secrets,Research question: How does metformin affect c...,neurological
2,dementia and cholecystitis: organ interplay,Purpose: This randomized controlled study exam...,hepatorenal
3,The interpeduncular nucleus regulates nicotine...,Partial lesions were made with kainic acid in ...,neurological
4,guillain-barre syndrome pathways in leukemia,Hypothesis: statins improves stroke outcomes v...,neurological
5,Effects of suprofen on the isolated perfused r...,Although suprofen has been associated with the...,hepatorenal
6,atherosclerosis and lymphoma: vascular insights,Aim: To investigate aspirin effects on diabete...,cardiovascular
7,Potential therapeutic use of the selective dop...,The clinical utility of dopamine (DA) D1 recep...,neurological
8,The basal ganglia connection in epilepsy,Background: dementia affects cardiac patients ...,neurological
9,septum and peripheral artery disease: vascular...,Purpose: This observational study examined cal...,cardiovascular


In [4]:
rows, columns = df.shape
print(f"Number of rows: {rows}, Number of columns: {columns}")

Number of rows: 3565, Number of columns: 3


In [5]:
 df = df.copy()

## Extracción de características con NLP

Pasos a seguir:
1. Preprocesamiento del texto

- Limpieza: eliminar caracteres especiales, números innecesarios, URLs, referencias
- Normalización: convertir a minúsculas, manejar acentos y caracteres especiales
- Tokenización: dividir el texto en palabras/tokens individuales
- Eliminación de stopwords: quitar palabras comunes sin valor semántico ("el", "la", "de", "and", "the")
- Stemming/Lemmatización: reducir palabras a su raíz o forma base

2.  Extracción de características textuales
Métodos tradicionales:

- Bag of Words (BoW): frecuencia de palabras
- TF-IDF: Term Frequency - Inverse Document Frequency
- N-gramas: combinaciones de 2-3 palabras consecutivas

3. Entrenamiento modelos clasicos
4. Reducción de dimensionalidad
5. Re-entrenamiento


## Ejemplo de test con los pasos anteriores:

1. Tokenización:  
Dividir texto en palabras, oraciones o elementos pequeños. Ej: Convertir párrafo en lista de palabras.

In [6]:
import nltk

def tokenize_text(text):
    return nltk.word_tokenize(text)

In [7]:
abstract_example = df.iloc[0]['abstract']
print(abstract_example)

Adrenoleukodystrophy ( ALD ) is a genetically determined disorder associated with progressive central demyelination and adrenal cortical insufficiency . All affected persons show increased levels of saturated unbranched very-long-chain fatty acids , particularly hexacosanoate ( C26  0 ) , because of impaired capacity to degrade these acids . This degradation normally takes place in a subcellular organelle called the peroxisome , and ALD , together with Zellwegers cerebrohepatorenal syndrome , is now considered to belong to the newly formed category of peroxisomal disorders . Biochemical assays permit prenatal diagnosis , as well as identification of most heterozygotes . We have identified 303 patients with ALD in 217 kindreds . These patients show a wide phenotypic variation . Sixty percent of patients had childhood ALD and 17 % adrenomyeloneuropathy , both of which are X-linked , with the gene mapped to Xq28 . Neonatal ALD , a distinct entity with autosomal recessive inheritance and p

In [8]:
tokens = tokenize_text(abstract_example)
print(tokens)

['Adrenoleukodystrophy', '(', 'ALD', ')', 'is', 'a', 'genetically', 'determined', 'disorder', 'associated', 'with', 'progressive', 'central', 'demyelination', 'and', 'adrenal', 'cortical', 'insufficiency', '.', 'All', 'affected', 'persons', 'show', 'increased', 'levels', 'of', 'saturated', 'unbranched', 'very-long-chain', 'fatty', 'acids', ',', 'particularly', 'hexacosanoate', '(', 'C26', '0', ')', ',', 'because', 'of', 'impaired', 'capacity', 'to', 'degrade', 'these', 'acids', '.', 'This', 'degradation', 'normally', 'takes', 'place', 'in', 'a', 'subcellular', 'organelle', 'called', 'the', 'peroxisome', ',', 'and', 'ALD', ',', 'together', 'with', 'Zellwegers', 'cerebrohepatorenal', 'syndrome', ',', 'is', 'now', 'considered', 'to', 'belong', 'to', 'the', 'newly', 'formed', 'category', 'of', 'peroxisomal', 'disorders', '.', 'Biochemical', 'assays', 'permit', 'prenatal', 'diagnosis', ',', 'as', 'well', 'as', 'identification', 'of', 'most', 'heterozygotes', '.', 'We', 'have', 'identified',

In [9]:
from nltk.tokenize import word_tokenize

2. Limpieza:
Eliminar caracteres especiales.

In [10]:
def clean_basic_tokens(tokens):
    return [token.lower() for token in tokens if token.isalpha()]

In [11]:
cleaned_tokens = clean_basic_tokens(tokens)
print(cleaned_tokens)

['adrenoleukodystrophy', 'ald', 'is', 'a', 'genetically', 'determined', 'disorder', 'associated', 'with', 'progressive', 'central', 'demyelination', 'and', 'adrenal', 'cortical', 'insufficiency', 'all', 'affected', 'persons', 'show', 'increased', 'levels', 'of', 'saturated', 'unbranched', 'fatty', 'acids', 'particularly', 'hexacosanoate', 'because', 'of', 'impaired', 'capacity', 'to', 'degrade', 'these', 'acids', 'this', 'degradation', 'normally', 'takes', 'place', 'in', 'a', 'subcellular', 'organelle', 'called', 'the', 'peroxisome', 'and', 'ald', 'together', 'with', 'zellwegers', 'cerebrohepatorenal', 'syndrome', 'is', 'now', 'considered', 'to', 'belong', 'to', 'the', 'newly', 'formed', 'category', 'of', 'peroxisomal', 'disorders', 'biochemical', 'assays', 'permit', 'prenatal', 'diagnosis', 'as', 'well', 'as', 'identification', 'of', 'most', 'heterozygotes', 'we', 'have', 'identified', 'patients', 'with', 'ald', 'in', 'kindreds', 'these', 'patients', 'show', 'a', 'wide', 'phenotypic',

3. Stemming:  
Llevar palabras a su forma raíz.

In [12]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
def stem_words(words):
    return [stemmer.stem(word) for word in words]

In [13]:
stemmed_tokens = stem_words(cleaned_tokens)
print(stemmed_tokens)


['adrenoleukodystrophi', 'ald', 'is', 'a', 'genet', 'determin', 'disord', 'associ', 'with', 'progress', 'central', 'demyelin', 'and', 'adren', 'cortic', 'insuffici', 'all', 'affect', 'person', 'show', 'increas', 'level', 'of', 'satur', 'unbranch', 'fatti', 'acid', 'particularli', 'hexacosano', 'becaus', 'of', 'impair', 'capac', 'to', 'degrad', 'these', 'acid', 'thi', 'degrad', 'normal', 'take', 'place', 'in', 'a', 'subcellular', 'organel', 'call', 'the', 'peroxisom', 'and', 'ald', 'togeth', 'with', 'zellweg', 'cerebrohepatoren', 'syndrom', 'is', 'now', 'consid', 'to', 'belong', 'to', 'the', 'newli', 'form', 'categori', 'of', 'peroxisom', 'disord', 'biochem', 'assay', 'permit', 'prenat', 'diagnosi', 'as', 'well', 'as', 'identif', 'of', 'most', 'heterozygot', 'we', 'have', 'identifi', 'patient', 'with', 'ald', 'in', 'kindr', 'these', 'patient', 'show', 'a', 'wide', 'phenotyp', 'variat', 'sixti', 'percent', 'of', 'patient', 'had', 'childhood', 'ald', 'and', 'adrenomyeloneuropathi', 'both'

4. Eliminar StopWords

In [14]:
def remove_stopwords(tokens):
    from nltk.corpus import stopwords
    academic_stopwords = {
        'abstract', 'paper', 'study', 'research', 'article', 'journal',
        'analysis', 'method', 'approach', 'technique', 'result', 'conclusion',
        'introduction', 'discussion', 'experimental', 'theoretical',
        'also', 'however', 'therefore', 'furthermore', 'moreover'
    }

    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words and token not in academic_stopwords]


In [15]:
filtered_tokens = remove_stopwords(stemmed_tokens)
print(filtered_tokens)

['adrenoleukodystrophi', 'ald', 'genet', 'determin', 'disord', 'associ', 'progress', 'central', 'demyelin', 'adren', 'cortic', 'insuffici', 'affect', 'person', 'show', 'increas', 'level', 'satur', 'unbranch', 'fatti', 'acid', 'particularli', 'hexacosano', 'becaus', 'impair', 'capac', 'degrad', 'acid', 'thi', 'degrad', 'normal', 'take', 'place', 'subcellular', 'organel', 'call', 'peroxisom', 'ald', 'togeth', 'zellweg', 'cerebrohepatoren', 'syndrom', 'consid', 'belong', 'newli', 'form', 'categori', 'peroxisom', 'disord', 'biochem', 'assay', 'permit', 'prenat', 'diagnosi', 'well', 'identif', 'heterozygot', 'identifi', 'patient', 'ald', 'kindr', 'patient', 'show', 'wide', 'phenotyp', 'variat', 'sixti', 'percent', 'patient', 'childhood', 'ald', 'adrenomyeloneuropathi', 'gene', 'map', 'neonat', 'ald', 'distinct', 'entiti', 'autosom', 'recess', 'inherit', 'point', 'resembl', 'zellweg', 'syndrom', 'account', 'case', 'although', 'excess', 'brain', 'patient', 'ald', 'partial', 'dietari', 'origin

### Procesamiento de dataset:

In [16]:
def preprocessNPL(text):
    tokens = tokenize_text(text)
    clean_tokens = clean_basic_tokens(tokens)
    stemmed_tokens = stem_words(clean_tokens)
    filtered_tokens = remove_stopwords(stemmed_tokens)
    return filtered_tokens


df['tokens_abstract'] = df['abstract'].apply(preprocessNPL)
df['tokens_title'] = df['title'].apply(preprocessNPL)

In [17]:
df.head(10)

,title,abstract,group,tokens_abstract,tokens_title
0,Adrenoleukodystrophy: survey of 303 cases: bio...,Adrenoleukodystrophy ( ALD ) is a genetically ...,neurological|hepatorenal,"[adrenoleukodystrophi, ald, genet, determin, d...","[adrenoleukodystrophi, survey, case, biochemis..."
1,endoscopy reveals ventricular tachycardia secrets,Research question: How does metformin affect c...,neurological,"[question, doe, metformin, affect, cancer, pit...","[endoscopi, reveal, ventricular, tachycardia, ..."
2,dementia and cholecystitis: organ interplay,Purpose: This randomized controlled study exam...,hepatorenal,"[purpos, thi, random, control, studi, examin, ...","[dementia, cholecyst, organ, interplay]"
3,The interpeduncular nucleus regulates nicotine...,Partial lesions were made with kainic acid in ...,neurological,"[partial, lesion, made, kainic, acid, interped...","[interpeduncular, nucleu, regul, nicotin, effe..."
4,guillain-barre syndrome pathways in leukemia,Hypothesis: statins improves stroke outcomes v...,neurological,"[hypothesi, statin, improv, stroke, outcom, vi...","[syndrom, pathway, leukemia]"
5,Effects of suprofen on the isolated perfused r...,Although suprofen has been associated with the...,hepatorenal,"[although, suprofen, ha, associ, develop, acut...","[effect, suprofen, isol, perfus, rat, kidney]"
6,atherosclerosis and lymphoma: vascular insights,Aim: To investigate aspirin effects on diabete...,cardiovascular,"[aim, investig, aspirin, effect, diabet, pe, a...","[atherosclerosi, lymphoma, vascular, insight]"
7,Potential therapeutic use of the selective dop...,The clinical utility of dopamine (DA) D1 recep...,neurological,"[clinic, util, dopamin, da, receptor, agonist,...","[potenti, therapeut, use, select, dopamin, rec..."
8,The basal ganglia connection in epilepsy,Background: dementia affects cardiac patients ...,neurological,"[background, dementia, affect, cardiac, patien...","[basal, ganglia, connect, epilepsi]"
9,septum and peripheral artery disease: vascular...,Purpose: This observational study examined cal...,cardiovascular,"[purpos, thi, observ, studi, examin, calcium, ...","[septum, peripher, arteri, diseas, vascular, i..."


In [140]:
dfstr=df.copy()

In [141]:
dfstr['tokens_abstract'] = df['tokens_abstract'].apply(lambda x: " ".join(x))
dfstr['tokens_title'] = df['tokens_title'].apply(lambda x: " ".join(x))

In [20]:
dfstr.head(10)

,title,abstract,group,tokens_abstract,tokens_title
0,Adrenoleukodystrophy: survey of 303 cases: bio...,Adrenoleukodystrophy ( ALD ) is a genetically ...,neurological|hepatorenal,adrenoleukodystrophi ald genet determin disord...,adrenoleukodystrophi survey case biochemistri ...
1,endoscopy reveals ventricular tachycardia secrets,Research question: How does metformin affect c...,neurological,question doe metformin affect cancer pituitari...,endoscopi reveal ventricular tachycardia secret
2,dementia and cholecystitis: organ interplay,Purpose: This randomized controlled study exam...,hepatorenal,purpos thi random control studi examin statin ...,dementia cholecyst organ interplay
3,The interpeduncular nucleus regulates nicotine...,Partial lesions were made with kainic acid in ...,neurological,partial lesion made kainic acid interpeduncula...,interpeduncular nucleu regul nicotin effect activ
4,guillain-barre syndrome pathways in leukemia,Hypothesis: statins improves stroke outcomes v...,neurological,hypothesi statin improv stroke outcom via migr...,syndrom pathway leukemia
5,Effects of suprofen on the isolated perfused r...,Although suprofen has been associated with the...,hepatorenal,although suprofen ha associ develop acut renal...,effect suprofen isol perfus rat kidney
6,atherosclerosis and lymphoma: vascular insights,Aim: To investigate aspirin effects on diabete...,cardiovascular,aim investig aspirin effect diabet pe analysi ...,atherosclerosi lymphoma vascular insight
7,Potential therapeutic use of the selective dop...,The clinical utility of dopamine (DA) D1 recep...,neurological,clinic util dopamin da receptor agonist treatm...,potenti therapeut use select dopamin receptor ...
8,The basal ganglia connection in epilepsy,Background: dementia affects cardiac patients ...,neurological,background dementia affect cardiac patient wor...,basal ganglia connect epilepsi
9,septum and peripheral artery disease: vascular...,Purpose: This observational study examined cal...,cardiovascular,purpos thi observ studi examin calcium channel...,septum peripher arteri diseas vascular insight


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# Vectorizar textos
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dfstr["tokens_abstract"])   
y = dfstr["group"]

# Split train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LinearSVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [194]:
import numpy as np
import lightgbm as lgb
from deap import base, creator, tools, algorithms
import random
import xgboost as xgb

In [195]:
# vectorizador para tokens de títulos (cambia valores string a matriz sparse: númerica)
tfidf_title = TfidfVectorizer(max_features=5000)#Ajustable
X_title = tfidf_title.fit_transform(dfstr['tokens_title'])

# vectorizador para tokens de abstracts
tfidf_abstract = TfidfVectorizer(max_features=10000)
X_abstract = tfidf_abstract.fit_transform(dfstr['tokens_abstract'])

In [196]:
top_tokens = np.argsort(np.array(X_abstract.sum(axis=0)).ravel())[-5000:]
Xcarac = X_abstract[:, top_tokens]
feature_names = np.array(tfidf_abstract.get_feature_names_out())[top_tokens]#Asigna nombres a las columnas con las que se entrena el modelo LGBM
                    #la posición del array indica el índice de Xcarac(subset de tokens más repetidos)
y = df['group']  #etiquetas

In [197]:
n_features = Xcarac.shape[1]

In [198]:
X_train, X_test, y_train, y_test = train_test_split(Xcarac, y, test_size=0.2, random_state=42, stratify=y)

In [200]:
def fitness_function(individual):
    selected_indices = [i for i, bit in enumerate(individual) if bit == 1]#Lista de 1 y 0s que indican token relevante a entrenar.
    if len(selected_indices) == 0:
        return 0.,  # evitar entrenamiento con 0 características

    clf = lgb.LGBMClassifier(
        n_estimators=50,
        max_depth=5,
        min_child_samples=5,
        min_gain_to_split=0.0,
        n_jobs=-1
    )

    #sparse matrix (n_muestras, n_features). Toma las características de acuerdo a los índices(col) indicados; conserva número de filas
    X_train_sel = X_train[:, selected_indices]
    X_test_sel  = X_test[:, selected_indices]

    #LightGBM: acepta matrices sparse
    clf.fit(X_train_sel, y_train, feature_name=[feature_names[i] for i in selected_indices])

    acc = clf.score(X_test_sel, y_test)#accuracy
    return acc,

In [ ]:
# Configuración DEAP
# Crear clase de fitness y individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=n_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
pop_size = 10
n_gen = 5

population = toolbox.population(n=pop_size)

algorithms.eaSimple(
    population,
    toolbox,
    cxpb=0.5,
    mutpb=0.2,
    ngen=n_gen,
    verbose=True
)

#Mejor individuo
best_ind = tools.selBest(population, 1)[0]
selected_features_final = [feature_names[i] for i, bit in enumerate(best_ind) if bit == 1]#tokens más significativos según el módelo

print("Número de tokens seleccionados:", len(selected_features_final))
print("Tokens seleccionados:", selected_features_final)

In [ ]:
# selected_features: tokens que DEAP seleccionó
vocab = set(vectorizer.get_feature_names_out())
valid_tokens = [t for t in selected_features if t in vocab]
print(len(valid_tokens))
print(len(selected_features))

In [ ]:
token_to_index = {t: i for i, t in enumerate(vectorizer.get_feature_names_out())}#Crea un diccionario con todos los tokens que el vectorizador(TF-IDF) identificó.
selected_indices = [token_to_index[t] for t in valid_tokens]#Asigna un índice a cada token. 

X_final = X_abstract[:, selected_indices]
print("Tamaño de la matriz final:", X_final.shape)

In [205]:
from sklearn.preprocessing import OneHotEncoder
original_columns = ['title', 'abstract']

# One-Hot Encoding y mantener sparse
encoder = OneHotEncoder(handle_unknown='ignore')
X_original_sparse = encoder.fit_transform(dfstr[original_columns])

In [206]:
from scipy.sparse import hstack

X_total = hstack([X_final, X_original_sparse])

In [ ]:
#Clasificación con tokens seleccionados y columnas title y abstract
X_ = X_total
y_ = dfstr["group"]
X_train, X_test, y_train, y_test = train_test_split(
    X_, y_, test_size=0.2, random_state=42, stratify=y
)

clf = lgb.LGBMClassifier(n_estimators=100, max_depth=7, n_jobs=-1)
clf.fit(X_train, y_train)
acc = clf.score(X_test, y_test)
print("Accuracy:", acc)

In [ ]:
# Crear columna unificada
dfstr["text"] = dfstr["title"] + " " + dfstr["abstract"] + " " + dfstr["tokens_abstract"] +" " + dfstr["tokens_title"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dfstr["text"])   # <- matriz numérica   # <- dataset con strings
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(dfstr["group"])  

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Clasificador Random Forest

In [182]:
x_train, x_test, y_train, y_test = train_test_split(
    Xcarac, y, test_size=0.2, random_state=42
)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

Clasificador XGBoost

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Separar las etiquetas por '|'
y_multilabel = [y.split('|') for y in dfstr['group']]

mlb = MultiLabelBinarizer()#transformar cada etiqueta en una columna 0/1
Y = mlb.fit_transform(y_multilabel)

In [192]:
x_train, x_test, y_train, y_test = train_test_split(
    Xcarac, Y, test_size=0.2, random_state=42
)

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10,
                            seed=123)
clf_xgb.fit(x_train, y_train)
preds_xgb = clf_xgb.predict(x_test)

print(classification_report(y_test, preds_xgb))